In [1]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.layers.advanced_activations import SReLU

from sklearn.metrics import precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split



Using Theano backend.
Using gpu device 0: GeForce GTX TITAN Black (CNMeM is disabled, cuDNN 5004)


In [2]:
iris = load_iris()

print('target count: %s' % (np.unique(iris.target).size))

frame = pd.DataFrame(iris.data, columns=iris.feature_names)
frame.describe()

target count: 3


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [3]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, random_state=0)
#y_test = np_utils.to_categorical(y_test)

In [4]:
def quality_meter(model, X_test, y_test, history, categorical=True):
    predicted = model.predict(X_test)
    if categorical:
        predicted = np.argmax(predicted, axis=1)
    else:
        predicted = np.argmax(predicted, axis=1)
    print(predicted.shape, y_test.shape)
    print(predicted)
    print(y_test)
    
    precision = precision_score(y_pred=predicted, y_true=y_test, average='micro')
    recall = recall_score(y_pred=predicted, y_true=y_test, average='micro')
    conf_matrix = confusion_matrix(y_pred=predicted, y_true=y_test)

    print('precision ', precision)
    print('recall ', recall)
    print('\n')
    print(pd.DataFrame(conf_matrix))

    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper left')
    plt.show()

In [5]:
#binary crossentropy model
def bc_model():
    model = Sequential()
    model.add(Dense(4, input_dim=4, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(3, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

#categorical crossentropy model
def cc_model():
    model = Sequential()
    model.add(Dense(4, input_dim=4, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [6]:
bc_model = bc_model()
cc_model = cc_model()

In [7]:
y_train_c = np_utils.to_categorical(y_train)
history = cc_model.fit(X_train, y_train_c, verbose=0, batch_size=135, nb_epoch=2000)

In [8]:
quality_meter(cc_model, X_test, y_test, history, categorical=True)

(15,) (15,)
[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1]
[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1]
precision  1.0
recall  1.0


   0  1  2
0  3  0  0
1  0  8  0
2  0  0  4


In [10]:
history1 = bc_model.fit(X_train, y_train_c, verbose=0, batch_size=32, nb_epoch=2000)

In [11]:
quality_meter(bc_model, X_test, y_test, history1, categorical=False)

(15,) (15,)
[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1]
[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1]
precision  1.0
recall  1.0


   0  1  2
0  3  0  0
1  0  8  0
2  0  0  4
